# Ideas:
- Добавить данные о карантине
- Доавить данные о карантине и принятых мерах
- Добавить данные о перелетах
- Добавить данные по завозу тестов в страну

Used datasets:
- https://www.kaggle.com/fernandol/countries-of-the-world
- https://www.kaggle.com/theworldbank/world-development-indicators
- https://www.kaggle.com/hbfree/covid19formattedweatherjan22march24
- https://github.com/tyz910/sberbank-covid19/blob/master/quarantine_dates.csv

In [0]:
debug = False
last_test_day = '2020/04/30' if debug else '2020/12/31'

# Imports and updates


In [0]:
from google.colab import drive
from IPython.display import clear_output
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/kaggle/covid_forecast')
clear_output()
import warnings
warnings.filterwarnings('ignore')

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import Ridge
from scipy.optimize import curve_fit
import json
import lightgbm as lgb

In [0]:
# make upgrade daily
update = False
if update:
    !rm -rf COVID-19
    !rm -rf COVID-19-web
    !git clone https://github.com/CSSEGISandData/COVID-19.git
    !git clone --single-branch --branch web-data https://github.com/CSSEGISandData/COVID-19.git COVID-19-web

In [0]:
def male(y, yhat):
    err = np.mean(np.absolute(np.log10((yhat + 1) / (y + 1))))
    return ('male', err, False)

# Open and merge datasets, look at data

### World development index to rate country situation

In [0]:
wdi_features = [
    'Country Code',
    "Air transport, passengers carried",
    "Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)",
    "Cause of death, by non-communicable diseases (% of total)",
    "Current health expenditure per capita, PPP (current international $)",
    "Death rate, crude (per 1,000 people)",
    "Diabetes prevalence (% of population ages 20 to 79)",
    "GDP per capita, PPP (current international $)",
    "Hospital beds (per 1,000 people)",
    "Incidence of tuberculosis (per 100,000 people)",
    "International migrant stock, total",
    "International tourism, number of arrivals",
    "International tourism, number of departures",
    "Labor force participation rate, total (% of total population ages 15+) (modeled ILO estimate)",
    "Life expectancy at birth, total (years)",
    "Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70 (%)",
    "Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population)",
    "Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population)",
    "Mortality rate, adult, female (per 1,000 female adults)",
    "Mortality rate, adult, male (per 1,000 male adults)",
    "Number of people spending more than 10% of household consumption or income on out-of-pocket health care expenditure",
    "Number of people spending more than 25% of household consumption or income on out-of-pocket health care expenditure",
    "Out-of-pocket expenditure (% of current health expenditure)",
    "People using at least basic sanitation services (% of population)",
    "People using safely managed sanitation services (% of population)",
    "People with basic handwashing facilities including soap and water (% of population)",
    "PM2.5 air pollution, population exposed to levels exceeding WHO guideline value (% of total)",
    "Population ages 15-64 (% of total)",
    "Population ages 65 and above (% of total)",
    "Population density (people per sq. km of land area)",
    "Population in the largest city (% of urban population)",
    "Population in urban agglomerations of more than 1 million (% of total population)",
    "Population, total",
    "Poverty headcount ratio at $3.20 a day (2011 PPP) (% of population)",
    "Prevalence of HIV, total (% of population ages 15-49)",
    "Smoking prevalence, females (% of adults)",
    "Smoking prevalence, males (% of adults)",
    "Survival to age 65, female (% of cohort)",
    "Survival to age 65, male (% of cohort)",
    "Trade (% of GDP)",
    "Tuberculosis case detection rate (%, all forms)",
    "Tuberculosis treatment success rate (% of new cases)",
    "Urban population (% of total)"
]

def read_wdi(fp='wdi-csv-zip-57-mb-/WDIData.csv'):
    wdi = pd.read_csv(fp)
    wdi = wdi.pivot(index='Country Code', columns='Indicator Name', values='2016').reset_index()[wdi_features]
    return wdi

wdi = read_wdi()

### Dataset with quarantine, medianage, beds in hospital

In [0]:
actions_taken = pd.read_csv('covid_dataset.csv').replace({-999: np.nan})

'''for this criterias max value represents the situation better'''
maxes = actions_taken.groupby('Country/Region')[['quarantine', 'schools', 'restrictions', 'tests', 'testpop']].max().reset_index()
'''for this criterias mean value represents the situation better'''
means = actions_taken.groupby('Country/Region')[['temperature', 'hospibed', 'medianage']].mean().reset_index()

actions_taken = pd.merge(maxes, means, on='Country/Region')

at_features = actions_taken.columns[1:].to_list()

### Additional info about countries

In [0]:
add_countries_info = pd.read_csv('countries of the world.csv')
add_countries_features = ['Country', 'Region', 'Net migration', 'Infant mortality (per 1000 births)', 'GDP ($ per capita)']
add_countries_info = add_countries_info[add_countries_features]
add_countries_info['Country'] = add_countries_info['Country'].str.strip()
for f in add_countries_features[2:-1]:
    add_countries_info[f] = add_countries_info[f].str.replace(',', '.').astype(float)

### Quarantine info

In [0]:
quar = pd.read_csv('quarantine_dates.csv')
quar.replace({'United States': 'US'}, inplace=True)
quar['Start date'] = pd.to_datetime(quar['Start date'])
quar['End date'] = pd.to_datetime(quar['End date'])
quar = quar.drop_duplicates(['Country'])

### Datasets provided by some university


In [0]:
def merge_provinces(df):
    days = df.columns[4:]
    grouped = df.groupby('Country/Region')
    new_df = pd.concat([grouped[['Lat', 'Long']].mean(), grouped[days].sum()], axis=1)
    return new_df.reset_index()

def add_test(df):
    # make test part filled with nans
    last_train_day = df.columns[-1]

    first_test_day = pd.to_datetime(last_train_day) + pd.Timedelta(days=1)
    t_cols = pd.date_range(first_test_day, pd.to_datetime(last_test_day)).to_series().dt.strftime('%m/%d/%Y')
    test = pd.DataFrame(np.nan, index=df.index, columns=t_cols)
    test['Country/Region'] = df['Country/Region']

    return pd.merge(df, test, on='Country/Region')

In [0]:
# read given data
ss = pd.read_csv('sample_submission_dDoEbyO.csv')
countries = pd.read_csv('countries.csv').dropna(subset=['iso_alpha2'])
confirmed = pd.read_csv('COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv').pipe(merge_provinces)
dead = pd.read_csv('COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv').pipe(merge_provinces)
recovered = pd.read_csv('COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv').pipe(merge_provinces)

# use it later for train test split
last_train_day = pd.to_datetime('2020/04/12')
# last_train_day = pd.to_datetime(confirmed.columns[-1]) + pd.Timedelta(days=1)

# add test
confirmed = confirmed.pipe(add_test)
recovered = recovered.pipe(add_test)
dead = dead.pipe(add_test)


# merge given countries info with obtained one
countries = pd.merge(countries, wdi, how='left', left_on='iso_alpha3', right_on='Country Code')
countries = pd.merge(countries, actions_taken, how='left', left_on='ccse_name', right_on='Country/Region')
countries = pd.merge(countries, add_countries_info, how='left', left_on='ccse_name', right_on='Country')

In [0]:
latest_data = pd.read_csv('COVID-19-web/data/cases_country.csv')
d = pd.to_datetime(latest_data['Last_Update']).max()
d = d.strftime('%m/%d/%Y')

latest_data = latest_data.set_index('Country_Region').loc[confirmed['Country/Region']].reset_index()

confirmed[d] = latest_data['Confirmed']
dead[d] = latest_data['Deaths']
recovered[d] = latest_data['Recovered']

In [0]:
len(set(list(actions_taken['Country/Region'])).intersection(set(list(countries['ccse_name']))))

In [0]:
len(set(list(add_countries_info['Country'])).intersection(set(list(countries['ccse_name']))))

In [0]:
len(set(list(wdi['Country Code'])).intersection(set(list(countries['iso_alpha3']))))

In [0]:
len(set(list(quar['Country'])).intersection(set(list(countries['Country/Region']))))

# Now you see?


In [0]:
def plot_ts(x, x_axis=None, labels=None):
    # plot time series
    if labels is None:
        labels = list(range(len(x)))
    fig = px.line()
    for i in range(len(x)):
        fig.add_trace(go.Scatter(
                y=x[i],
                x=x_axis,
                name=f"{labels[i]}",
                opacity=0.8))
    fig.show()

In [0]:
plot_ts(confirmed[confirmed.columns[4:]].values, x_axis=confirmed.columns[4:], labels=confirmed['Country/Region'].values)

In [0]:
plot_ts(dead[dead.columns[4:]].values, x_axis=dead.columns[4:], labels=dead['Country/Region'].values)

# Feature engeneering

### Quarantine and country features


In [0]:
num_features = []
cat_features = []
targets = ['confirmed', 'dead', 'recovered']

In [0]:
# melt and merge everyting
def melt(df, value_name='None'):
    # to not to write it 3 times
    melted = df.melt(id_vars=['Country/Region', 'Lat', 'Long'], var_name='date', value_name=value_name)
    melted['date'] = pd.to_datetime(melted['date'])
    return melted

# join recovered, dead and confirmed into single dataset
melted = pd.merge(melt(dead, 'dead'), melt(confirmed, 'confirmed'), on=['Country/Region', 'date'])
melted = pd.merge(melted, melt(recovered, 'recovered'), on=['Country/Region', 'date'])

In [0]:
melted = pd.merge(melted, quar, how='left', left_on='Country/Region', right_on='Country')

# add quarantine features
melted['days_since_quar_start'] = melted['date'] - melted['Start date']

# make delta float and remove negative deltas
melted['days_since_quar_start'] = melted['days_since_quar_start'] / np.timedelta64(1, 'D')
melted.loc[melted['days_since_quar_start'] < 0, 'days_since_quar_start'] = np.nan

melted.rename({'Level': 'quar_type'}, axis=1, inplace=True)

num_features += ['days_since_quar_start']
cat_features += ['quar_type']

# add countries features
data = pd.merge(melted, countries, left_on='Country/Region', right_on='ccse_name', how='left')
data.rename({'Country/Region_x': 'Country/Region'}, axis=1, inplace=True)

In [0]:
for t in targets:
    data[t] = np.maximum(0, data[t])

# important note: we are predicting delta, not cumulative sum
data.loc[:, targets] = data[targets] - data.groupby('Country/Region')[targets].shift(1)

for t in targets:
    data[t] = np.maximum(0, data[t])

### Time based features

In [0]:
# if our max shift/rolling is  7 set days to 7
# used to not to recalculate all date statistics when predicting recursively
max_shift = 8
max_shift_delta = pd.Timedelta(days=max_shift)

In [0]:
def ridge_features(y):
    # returns Ridge params and predictions a week ahead
    pred_day = 8

    if (y.isna().any()) or (y == 0).all():
        return pd.Series(np.zeros((3,)), index=['ridge_bias', 'ridge_coef', 'ridge_pred'])        
    x = np.arange(1, len(y) + 1).reshape(len(y), -1)
    y = y[::-1]
    r = Ridge()
    r.fit(x, y)
    pred = r.predict([[pred_day]])[0]
    return pd.Series([r.coef_[0], r.intercept_, pred], index=['ridge_bias', 'ridge_coef', 'ridge_pred'])

def poly_features(y):
    # fit polynomial to data, to better see growth
    # returns poly coeffs and pred
    deg = 3  # polynomial up to deg order
    pred_day = 8 # day we make predictions for
    cnames = [f'poly_{i}' for i in range(deg + 1)] + ['poly_pred']
    if (y.isna().any()) or (y == 0).all():
        return pd.Series(np.zeros((len(cnames),)), index=cnames)
    x = np.arange(1, len(y) + 1)
    y = y[::-1]
    params = np.polyfit(x, y, deg)[::-1]
    pred = np.polyval(params[::-1], [pred_day])
    return pd.Series(np.append(params, pred), index=cnames)

def add_time_features(data):
    num_features = []
    cat_features = []
    
    # add lag features
    lags = [1, 2, 3, 4, 5, 6, 7]
    for lag in lags:
        lag_features = data.groupby('Country/Region')[['confirmed', 'dead', 'recovered']].shift(lag)
        lag_features.columns = [f'{col}_lag_{lag}' for col in lag_features.columns]
        num_features += lag_features.columns.to_list()
        data.drop(lag_features.columns, axis=1, inplace=True, errors='ignore')
        data = pd.concat([data, lag_features], axis=1)

    # rolling statistics
    window = 7
    rollings = data.groupby('Country/Region')[['confirmed', 'dead', 'recovered']].shift(1).rolling(window).agg(['mean', 'std', 'max', 'min'])
    rollings.columns = ['rolling_' + '_'.join(i) for i in rollings.columns]
    num_features += rollings.columns.to_list()
    data.drop(rollings.columns, axis=1, inplace=True, errors='ignore')
    data = pd.concat([data, rollings], axis=1)

    # max target delta expanding
    for t in targets:
        data[t + '_expanding_max'] = data.groupby('Country/Region')[t].transform(lambda x: x.shift(1).expanding().max())
        num_features += [t + '_expanding_max']

    #  max_confirmed - lag1_confirmed, max_confirmed - lag2_confirmed

    lag_minus_max_features = ['lag1_minus_max', 'lag2_minus_max', 'lag3_minus_max']
    lag_minus_max = [1, 2, 3]
    for l, col_name in zip(lag_minus_max, lag_minus_max_features):
        data[col_name] = data['confirmed_expanding_max'] - data[f'confirmed_lag_{l}']
    num_features += lag_minus_max_features

    if True:
        # ridge regression and polynomial on lags for confirmed, recovered and dead
        for t in ['confirmed_', 'recovered_', 'dead_']:
            t_lags = data.columns[data.columns.str.contains(t + 'lag_')]
            rfs = data[t_lags].apply(ridge_features, axis=1, result_type='expand')
            rfs.columns = [t + i for i in rfs.columns]
            data.drop(rfs.columns, axis=1, inplace=True, errors='ignore')
            data = pd.concat([data, rfs], axis=1)
            num_features += rfs.columns.to_list()

            pfs = data[t_lags].apply(poly_features, axis=1, result_type='expand')
            pfs.columns = [t + i for i in pfs.columns]
            data.drop(pfs.columns, axis=1, inplace=True, errors='ignore')
            data = pd.concat([data, pfs], axis=1)
            num_features += pfs.columns.to_list()

    return data, num_features, cat_features

In [0]:
data, time_num_features, time_cat_features = add_time_features(data)

In [0]:
num_features += wdi_features[1:]  # first is country code
num_features += ['Net migration', 'Infant mortality (per 1000 births)', 'GDP ($ per capita)']  # add info features
cat_features += ['Region']
num_features += at_features  # actions taken features
num_features += time_num_features
cat_features += time_cat_features

num_features = list(set(num_features))
cat_features = list(set(cat_features))

features = num_features + cat_features
len(features)

In [0]:
# early days don't contain much info and we don't need nan targets in train
data = data[
    data['date'] >= '02/01/2020'
]
data
# fill nans and encode cat features
data.loc[:, cat_features] = data[cat_features].fillna('None')
data.loc[:, num_features] = data.loc[:, num_features].fillna(0)

for cat_f in cat_features:
    cat_en = LabelEncoder()
    data[cat_f] = cat_en.fit_transform(data[cat_f])

# Let's try to fit something

In [0]:
print(last_train_day)

In [0]:
def fit_lgb(train):
    # features are finetuned in a separate kernel
    booster_params = {
        'confirmed': {
            'objective': 'poisson', 'boosting_type': 'gbdt', 'n_jobs': -1, 'seed': 2, 'num_iterations': 1500, 'learning_rate': 0.03, 'num_leaves': 127, 
            'min_data_in_leaf': 35, 'min_gain_to_split': 0.1, 'bagging_fraction': 0.9, 'bagging_freq': 1, 'feature_fraction': 1.0, 'lambda_l2': 0.0
        }, 
        'dead': {
            'objective': 'poisson', 'boosting_type': 'gbdt', 'n_jobs': -1, 'seed': 2, 'num_iterations': 1500, 'learning_rate': 0.03, 'num_leaves': 127, 
            'min_data_in_leaf': 25, 'min_gain_to_split': 0.0, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 1.0, 'lambda_l2': 0.05
        }, 
        'recovered': {
            'objective': 'regression', 'boosting_type': 'gbdt', 'n_jobs': -1, 'seed': 2, 'num_iterations': 1500, 'learning_rate': 0.01, 'num_leaves': 15, 
            'min_data_in_leaf': 35, 'min_gain_to_split': 0.1, 'bagging_fraction': 0.8, 'bagging_freq': 5, 'feature_fraction': 0.8, 'lambda_l2': 0.0
        }
    }
    boosters = {}
    X_train = train[features]
    for t in targets:
        y_train = train[t]
        model = lgb.LGBMRegressor(**booster_params[t])
        print(f"Fitting {t}'s model")
        model.fit(
            X_train, y_train,
            eval_metric=male,
            eval_set=[(X_train, y_train)],
            verbose=100,
            categorical_feature=cat_features
        )
        print()
        boosters[t] = model
    return boosters

In [0]:
def run_lgb(models, test):
    preds = pd.DataFrame()
    for key in models.keys():
        pred = np.maximum(0, models[key].predict(test[features]))
        pred = np.round(pred).astype('int')
        preds[key] = pred
    return preds

In [0]:
def fit_predict_recursively(data, train_last_day):
    data = data.copy()
    train = data[data['date'] <= train_last_day]
    test = data[data['date'] > train_last_day]

    models = fit_lgb(train)
    for i in range(1, int((test['date'].max() - train_last_day).days + 1)):
        pred_day = train_last_day + pd.Timedelta(days=i)
        print(pred_day)
        # recalculate time_features for pred_day
        recalc_window = (data['date'] >= pred_day - max_shift_delta) & (data['date'] <= pred_day)
        recalced = add_time_features(data.loc[recalc_window])[0]
        data.loc[data['date'] == pred_day, :] = recalced.loc[recalced['date'] == pred_day, :]
        xt = data[data['date'] == pred_day]
        data.loc[data['date'] == pred_day, targets] = run_lgb(models, xt).values

    return data


In [0]:
def crossval(data, h) -> (float, float):
    val_data = data[data['date'] <= last_train_day]
    val_last_train_day = last_train_day - pd.Timedelta(days=h)

    val_preds = fit_predict_recursively(val_data, val_last_train_day)
    val_preds = val_preds[val_preds['date'] > val_last_train_day]
    val_real = val_data[val_data['date'] > val_last_train_day]
    return male(val_real['confirmed'], val_preds['confirmed'])[1], male(val_real['dead'], val_preds['dead'])[1]

In [0]:
crossval(data, 7)

# Visualize validationpredictions


In [0]:
pdata = fit_predict_recursively(data, last_train_day)

In [0]:
def pivot_cum(df, target):
    pdf = df.pivot('Country/Region', 'date', target)
    pdf.columns = pdf.columns.to_series().dt.strftime('%m/%d/%Y')
    pdf = pdf.cumsum(axis=1)
    return pdf.reset_index()

In [0]:
# stands for pivot cumulated
confirmed_pred = pivot_cum(pdata, 'confirmed')
confirmed_pred = confirmed_pred.sort_values(by=confirmed_pred.columns[-1], ascending=False)

dead_pred = pivot_cum(pdata, 'dead')
dead_pred = dead_pred.sort_values(by=dead_pred.columns[-1], ascending=False)

In [0]:
plot_ts(confirmed_pred.iloc[:, 1:].values, x_axis=confirmed_pred.iloc[:, 1:].columns, labels=confirmed_pred['Country/Region'].values)

In [0]:
plot_ts(dead_pred.iloc[:, 1:].values, x_axis=dead_pred.iloc[:, 1:].columns, labels=dead_pred['Country/Region'].values)

# Write submission

In [0]:
def to_csv(conf_pred, dead_pred, fp='subs/covid_preds.csv'):
    # process and write to csv
    cols_order = ss.columns
    conf_pred = pd.merge(conf_pred, countries[['ccse_name', 'iso_alpha3']], left_on='Country/Region', right_on='ccse_name').drop(['Country/Region', 'ccse_name'], axis=1).melt('iso_alpha3', var_name='date', value_name='confirmed')
    dead_pred = pd.merge(dead_pred, countries[['ccse_name', 'iso_alpha3']], left_on='Country/Region', right_on='ccse_name').drop(['Country/Region', 'ccse_name'], axis=1).melt('iso_alpha3', var_name='date', value_name='dead')

    mysub = pd.merge(conf_pred, dead_pred, on=['iso_alpha3', 'date'])
    mysub['date'] = pd.to_datetime(mysub['date'])
    mysub = mysub[mysub['date'] >= '2020-04-05']
    mysub['date'] = mysub['date'].dt.strftime('%Y-%m-%d')
    mysub = mysub.rename({'iso_alpha3': 'country'}, axis=1).reset_index(drop=True)
    mysub = pd.merge(ss, mysub, on=['date', 'country'], how='left')
    mysub.drop(['prediction_confirmed', 'prediction_deaths'], axis=1, inplace=True)
    mysub.rename({'confirmed': 'prediction_confirmed', 'dead': 'prediction_deaths'}, axis=1, inplace=True)
    mysub = mysub[cols_order]
    print(mysub.isna().sum().to_string())
    mysub['prediction_confirmed'] = mysub['prediction_confirmed'].fillna(18)
    mysub['prediction_deaths'] = mysub['prediction_deaths'].fillna(0)
    print(mysub.isna().sum().to_string())
    mysub['prediction_confirmed'] = mysub['prediction_confirmed'].astype(int)
    mysub['prediction_deaths'] = mysub['prediction_deaths'].astype(int)

    mysub.to_csv(fp, index=False)

to_csv(confirmed_pred, dead_pred, 'subs/latest_data.csv')

In [3]:
pd.read_csv('subs/latest_data.csv')

,date,country,prediction_confirmed,prediction_deaths
0,2020-04-05,AFG,349,7
1,2020-04-06,AFG,367,11
2,2020-04-07,AFG,423,14
3,2020-04-08,AFG,444,14
4,2020-04-09,AFG,484,15
...,...,...,...,...
45794,2020-12-27,ZWE,14,3
45795,2020-12-28,ZWE,14,3
45796,2020-12-29,ZWE,14,3
45797,2020-12-30,ZWE,14,3
